# Linking to Spotify

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

secrets_file = open("secrets.txt","r")
string = secrets_file.read()

secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
        
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

# Preparing the library

In [2]:
from random import randint
import pandas as pd
from pandas import json_normalize

pop_100 = pd.read_csv('pop_100.csv').drop('Unnamed: 0', axis = 1)

song_library = pd.read_csv('song_library.csv').drop('Unnamed: 0', axis = 1)
song_library = song_library[song_library['name'].isna()==False].reset_index(drop=True)

X_num = song_library[['danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_num)
X_prep = scaler.transform(X_num)

In [4]:
from sklearn import cluster
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=9, random_state=1234, n_init=10) # k=9, found earlier
kmeans.fit(X_prep)

clusters = kmeans.predict(X_prep)

song_library['category'] = clusters

# User chooses a song

In [20]:
user_song = input('Choose a song: ')

Choose a song: I Giorni


# Song generator

In [23]:
if user_song in list(pop_100['Song']):
    rand_int = randint(0,len(pop_100))
    print('Alternative song: ',pop_100['Song'][rand_int])
    print('By:               ',pop_100['Artist'][rand_int])
    
else:
    search_results = sp.search(q=user_song, limit=3)
    user_song_options = json_normalize(search_results['tracks']['items'])
    user_song_id = user_song_options['id'][0]
    
    print('Your choice : ',user_song_options['name'][0])
    print('By          : ',user_song_options['artists'][0][0]['name'])
    print('Album       : ',user_song_options['album.name'][0])
    print('url         : ','https://open.spotify.com/track/'+user_song_id)
    
    
    if user_song_id in list(song_library['song_id']):
        user_song_cat = song_library.loc[song_library['song_id'] == user_song_id,'category'].values[0]

        print('Category    : ',user_song_cat,'\n')

        rand_int = randint(0,len(song_library[song_library['category']==user_song_cat]))

        suggested_track = song_library[song_library['category']==user_song_cat].iloc[rand_int,0]
        suggested_artist = song_library[song_library['category']==user_song_cat].iloc[rand_int,1]
        suggested_url = song_library[song_library['category']==user_song_cat].iloc[rand_int,2]
        suggested_category = song_library[song_library['category']==user_song_cat].iloc[rand_int,21]

        print('Suggested track : ',suggested_track)
        print('By              : ',suggested_artist)
        print('url             : ','https://open.spotify.com/track/'+suggested_url)
        print('Category check  : ',suggested_category)
   
    else: # otherwise, find its audio_features and use kmean to predict a category
        user_song_audio = sp.audio_features(user_song_id)
        user_song_audio_df = json_normalize(user_song_audio)
        user_song_audio_df_num = user_song_audio_df[['danceability', 'energy', 'key',
           'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
           'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
        user_song_audio_prep = scaler.transform(user_song_audio_df_num)

        user_song_cat1 = kmeans.predict(user_song_audio_prep)[0]

        print('Predicted   : ',user_song_cat1,'\n')

        rand_int = randint(0,len(song_library[song_library['category']==user_song_cat1]))

        suggested_track = song_library[song_library['category']==user_song_cat1].iloc[rand_int,0]
        suggested_artist = song_library[song_library['category']==user_song_cat1].iloc[rand_int,1]
        suggested_url = song_library[song_library['category']==user_song_cat1].iloc[rand_int,2]
        suggested_category = song_library[song_library['category']==user_song_cat1].iloc[rand_int,21]

        print('Suggested track : ',suggested_track)
        print('By              : ',suggested_artist)
        print('url             : ','https://open.spotify.com/track/'+suggested_url)
        print('Category check  : ',suggested_category)

Your choice :  I Giorni
By          :  Ludovico Einaudi
Album       :  Kołysanki
url         :  https://open.spotify.com/track/1PGFtLsMJXaom0xQ0O6ZaG
Predicted   :  1 

Suggested track :  Glassworks: Opening
By              :  Víkingur Ólafsson
url             :  https://open.spotify.com/track/2XYEu7Deyz20WeL6Py3XLc
Category check  :  1
